In [1]:
"""
Script to extract recent game review from IGN
"""

'\nScript to extract recent game review from IGN\n'

In [2]:
import os
os.getcwd()
os.chdir("/Users/natepruitt/Projects/robocritic/")

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# Imports
import requests
from bs4 import BeautifulSoup
from helper_functions.db_connection import connection
from helper_functions.query_functions import get_publisher_by_id, get_game_by_title
from helper_functions.igdb_service import IGDBService

Connection to MySQL DB successful


In [5]:
# Pull latest games from IGN
ign_latest_review_url = "https://www.ign.com/reviews/games"
header_mapping = {"User-Agent": "Mac Firefox"}
publisher = get_publisher_by_id(connection, 1)

In [6]:
ign_latest_reviews_html = requests.get(ign_latest_review_url, headers=header_mapping)

In [7]:
ign_latest_reviews_soup = BeautifulSoup(ign_latest_reviews_html.content, 'html.parser')

In [8]:
latest_reviews_a_tag_list = ign_latest_reviews_soup.find(class_="main-content").find_all('a')

In [9]:
# From each review, need to extract review url and game name. 
# Or, only get the game name. I already have the URL from the database. Add review to it. I know the game name start at 1 and repeat to 29
# 'get' soup function returns attribute on a soup object. 
latest_reviews_a_tag_list[0].get("href")


'/articles/avatar-frontiers-of-pandora-review'

In [10]:
len(latest_reviews_a_tag_list)

30

In [11]:
bool(latest_reviews_a_tag_list[1].get('aria-label'))

False

In [12]:
filter_latest_reviews_a_tag = filter(lambda x: bool(x.get('aria-label')), latest_reviews_a_tag_list)

In [13]:
filtered_a_tags = list(filter_latest_reviews_a_tag)

In [14]:
filtered_a_tags[0].get('aria-label').rstrip('Review').rstrip()

'Avatar: Frontiers of Pandora'

In [15]:
filtered_a_tags[0].get('href')

'/articles/avatar-frontiers-of-pandora-review'

In [16]:
ign_base_url = "https://www.ign.com"

In [17]:
avatar_article_link = ign_base_url + filtered_a_tags[0].get('href')

In [18]:
avatar_article_link

'https://www.ign.com/articles/avatar-frontiers-of-pandora-review'

In [19]:
mapped_latest_reviews = list(map(lambda x: {"title": x.get("aria-label").rstrip("Review").rstrip(), "href": x.get("href")}, filtered_a_tags))


In [20]:
mapped_latest_reviews

[{'title': 'Avatar: Frontiers of Pandora',
  'href': '/articles/avatar-frontiers-of-pandora-review'},
 {'title': 'Arizona Sunshine 2',
  'href': '/articles/arizona-sunshine-2-review'},
 {'title': 'Lethal Company Early Access',
  'href': '/articles/lethal-company-early-access-review'},
 {'title': 'The Walking Dead: Destinies',
  'href': '/articles/the-walking-dead-destinies-review'},
 {'title': 'SteamWorld Build', 'href': '/articles/steamworld-build-review'},
 {'title': 'Bluey: The Videogame',
  'href': '/articles/bluey-the-videogame-review'},
 {'title': 'Flashback 2', 'href': '/articles/flashback-2-review'},
 {'title': 'Naruto x Boruto Ultimate Ninja Storm Connections',
  'href': '/articles/naruto-x-boruto-ultimate-ninja-storm-connections-review'},
 {'title': 'Modern Warfare 3: Zombies Final',
  'href': '/articles/call-of-duty-modern-warfare-3-zombies-review'},
 {'title': "Assassin's Creed Nexus VR",
  'href': '/articles/assassins-creed-nexus-vr-review'}]

In [21]:
bool(get_game_by_title(connection, 'The Last Faith'))

True

In [22]:
get_game_by_title(connection, 'The Last Faith')

(1, 'The Last Faith', datetime.date(2023, 11, 15), None)

In [23]:
# For each game, write the name and date

In [24]:
from helper_functions.write_functions import create_game
from datetime import datetime

In [25]:
avatar_release_date = datetime.utcfromtimestamp(1701907200)
avatar_release_date.date()

datetime.date(2023, 12, 7)

In [26]:
create_game(connection, {"title": "Avatar: Frontiers of Pandora", "release_date": avatar_release_date.date()})

The error '1062 (23000): Duplicate entry 'Avatar: Frontiers of Pandora' for key 'game.Unique_Game'' occurred


In [35]:
get_game_by_title(connection, 'Avatar: Frontiers of Pandora')

(2,
 'Avatar: Frontiers of Pandora',
 datetime.date(2023, 12, 6),
 datetime.datetime(2023, 12, 7, 12, 6, 10))

In [5]:
# retrieve the game release date from igdb API
igdb_service = IGDBService(connection)

In [6]:
igdb_service.get_game_release_date_by_title('Avatar: Fronteirs of Pandora')

482972

In [41]:
igdb_service = IGDBService(connection)

In [42]:
print(igdb_service.get_game_release_date_by_title(' '))

[{'id': 152202, 'genres': [31], 'name': 'Avatar: Frontiers of Pandora', 'platforms': [6, 167, 169], 'release_dates': [482972, 482973, 482974]}]
